In [5]:
# install Libraries
%%capture
!pip install torch==2.5.0 transformers TTS langdetect

In [6]:
#Import Libraries

from TTS.api import TTS
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
from langdetect import detect
import io
import torch

`Download Models`

In [7]:
# TTS
try:
    tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to("cuda" if torch.cuda.is_available() else "cpu")
except Exception as e:
    raise RuntimeError(f"Failed to load TTS model: {str(e)}")


try:
    # model aubmindlab for arabic
    arabic_model_name = "aubmindlab/bert-base-arabertv02-twitter"
    sentiment_tokenizer = AutoTokenizer.from_pretrained(arabic_model_name)
    sentiment_model = AutoModelForSequenceClassification.from_pretrained("UBC-NLP/MARBERT")
except Exception as e:
    raise RuntimeError(f"Failed to load Arabic sentiment analysis model: {str(e)}")

# model distilbert for english
try:
    sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
except Exception as e:
    raise RuntimeError(f"Failed to load English sentiment analysis model: {str(e)}")


 > You must confirm the following:
 | > "I have purchased a commercial license from Coqui: licensing@coqui.ai"
 | > "Otherwise, I agree to the terms of the non-commercial CPML: https://coqui.ai/cpml" - [y/n]
 | | > y
 > Downloading model to /root/.local/share/tts/tts_models--multilingual--multi-dataset--xtts_v2


100%|█████████▉| 1.87G/1.87G [00:40<00:00, 41.2MiB/s]
100%|██████████| 1.87G/1.87G [00:40<00:00, 46.2MiB/s]
100%|██████████| 4.37k/4.37k [00:00<00:00, 16.5kiB/s]
 41%|████      | 148k/361k [00:00<00:00, 677kiB/s]
100%|██████████| 361k/361k [00:00<00:00, 743kiB/s]
100%|██████████| 32.0/32.0 [00:00<00:00, 111iB/s]


 > Model's license - CPML
 > Check https://coqui.ai/cpml.txt for more info.
 > Using model: xtts


/usr/local/lib/python3.11/dist-packages/TTS/tts/layers/xtts/xtts_manager.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.speakers = torch.load(speaker_file_path)
/u

tokenizer_config.json:   0%|          | 0.00/476 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/654M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cuda:0


In [8]:
# Converting feelings into words that are suitable for pronunciation
def map_sentiment_to_emotion(sentiment, language="en"):
    if language == "ar":
        # For arabic
        if sentiment == "positive" or sentiment == "POS":
            return "happy"
        elif sentiment == "negative" or sentiment == "NEG":
            return "sad"
        else:
            return "neutral"
    else:
        # For English
        if "positive" in sentiment.lower():
            return "happy"
        elif "negative" in sentiment.lower():
            return "sad"
        else:
            return "neutral"

#
def arabic_sentiment_analysis(text):
    # Use a simple method using Arabic keywords for feelings.
    positive_words = ["سعيد", "فرح", "ممتاز", "رائع", "جيد", "حب", "جميل", "نجاح", "أحسنت", "شكرا"]
    negative_words = ["حزين", "غاضب", "سيء", "فشل", "خطأ", "مشكلة", "صعب", "لا أحب", "سخيف", "مؤسف"]

    # Count positive and negative words in the text.
    positive_count = sum(1 for word in positive_words if word in text.lower())
    negative_count = sum(1 for word in negative_words if word in text.lower())

    # Determine sentiment based on word count
    if positive_count > negative_count:
        return "positive"
    elif negative_count > positive_count:
        return "negative"
    else:
        # Use more complex analysis when there are no clear words.
        try:
            # Another free template can be used if available.
            inputs = sentiment_tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
            with torch.no_grad():
                outputs = sentiment_model(**inputs)

            # Determine the category based on the resulting values.
            sentiment_class = torch.argmax(outputs.logits).item()

            # Convert to appropriate category (positive/negative/neutral)
            if sentiment_class == 0:
                return "negative"
            elif sentiment_class == 1:
                return "neutral"
            else:
                return "positive"
        except:
            #In case of failure, we consider the text neutral.
            return "neutral"

# Improved language detection
def detect_language_safely(text):
    try:
        # More accurate detection of Arabic language
        if any('\u0600' <= c <= '\u06FF' for c in text):
            return "ar"
        return detect(text)
    except Exception:
        # If failed, check for Arabic characters.
        if any('\u0600' <= c <= '\u06FF' for c in text):
            return "ar"
        return "en"




`Inference`

In [11]:

text = "i am ver happy , really"

# Better language detection
detected_language = detect_language_safely(text)
language = "ar" if detected_language == "ar" else "en"

# Sentiment analysis
emotion = "neutral"
sentiment_result = None

if language == "en":
    try:
        sentiment_result = sentiment_analyzer(text)[0]
        emotion = map_sentiment_to_emotion(sentiment_result["label"])
        print(f"English sentiment analysis result: {sentiment_result}")
    except Exception as e:
        print(f"Sentiment analysis failed: {str(e)}")
else:
    try:
        sentiment_result = arabic_sentiment_analysis(text)
        emotion = map_sentiment_to_emotion(sentiment_result, language="ar")
        print(f"Arabic sentiment analysis result: {sentiment_result}")
    except Exception as e:
        print(f"Arabic sentiment analysis failed: {str(e)}")


print(f"Sentiment analysis: {sentiment_result}")
print(f"Language detected: {language}")

# Specify the final audio file name.
output_filename = "output.wav"

#Create audio and save it to file
try:
    tts.tts_to_file(
        text=text,
        file_path=output_filename,
        emotion=emotion,
        speaker_wav="speaker.wav",
        language=language
    )
    print(f"✅ The audio was successfully created and saved in{output_filename}")
    print(f"🔊 Feelings discovered: {emotion.capitalize()}")
except Exception as e:
    print(f"❌ Failed to create audio: {str(e)}")

English sentiment analysis result: {'label': 'POSITIVE', 'score': 0.9993470311164856}
Sentiment analysis: {'label': 'POSITIVE', 'score': 0.9993470311164856}
Language detected: en
 > Text splitted to sentences.
['i am ver happy , really']


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


 > Processing time: 3.6873230934143066
 > Real-time factor: 1.2855026911488974
✅ The audio was successfully created and saved inoutput.wav
🔊 Feelings discovered: Happy


In [12]:
from IPython.display import Audio
Audio("output.wav")